## Parsing Core Subject Thesaurus

In [4]:
import os
import xml.etree.ElementTree as ET
import networkx as nx
from collections import defaultdict

In [5]:
data_dir = "/home/chris/directory/data/Thesaurus"

xml = "CST20160704.xml"

In [6]:
tree = ET.parse(os.path.join(data_dir, xml))

root = tree.getroot()

In [23]:
subject_categories = defaultdict(int)
descriptors = []
non_descriptors = []
relationships = []

for concept in root:
    
    data = []
    for child in concept:
        data.append((child.tag, child.text))
        
    descriptor = ""
    valid = True
        
    if data[0][0]  == "DESCRIPTOR":
        descriptor = data[0][1]
        
    elif data[0][0] == "NON-DESCRIPTOR":
        descriptor = data[0][1]
        valid = False
        
    for category, text in data:
        
        if category == "French":
            relationships.append((text, descriptor, "translation"))
        elif category == "Use":
            relationships.append((descriptor, text, "use"))
        elif category == "UsedFor":
            relationships.append((text, descriptor, "used for"))
        elif category == "BroaderTerm":
            relationships.append((descriptor, text, "broader term"))
        elif category == "NarrowerTerm":
            relationships.append((text, descriptor, "narrow term"))
        elif category == "RelatedTerm":
            relationships.append((text, descriptor, "related term"))
        elif category == "SubjectCategory":
            relationships.append((descriptor, text, "subject category"))
        else:
            pass
            


relationships

[('Abréviation', 'Abbreviations', 'translation'),
 ('Terminology', 'Abbreviations', 'related term'),
 ('Abbreviations', 'LN Language and Linguistics', 'subject category'),
 ('Abilities', 'Skills', 'use'),
 ('Abilities', 'PR Processes', 'subject category'),
 ('Affaires autochtones', 'Aboriginal affairs', 'translation'),
 ('Aboriginal issues', 'Aboriginal affairs', 'used for'),
 ('Aboriginal rights', 'Aboriginal affairs', 'related term'),
 ('Land claims', 'Aboriginal affairs', 'related term'),
 ('Aboriginal affairs', 'GV Government and Politics', 'subject category'),
 ('Aboriginal affairs', 'SO Society and Culture', 'subject category'),
 ('Entreprise autochtone', 'Aboriginal businesses', 'translation'),
 ('Aboriginal businesses', 'Businesses', 'broader term'),
 ('Aboriginal programs', 'Aboriginal businesses', 'related term'),
 ('Aboriginal businesses', 'EC Economics and Industry', 'subject category'),
 ('Culture autochtone', 'Aboriginal culture', 'translation'),
 ('Aboriginal heritage', 

In [29]:
G = nx.DiGraph()

In [30]:
for term1, term2, relationship in relationships:
    G.add_edge(term1, term2, rel_type=relationship)

In [31]:
nx.write_gexf(G, os.path.join(data_dir, 'core_thesaurus.gexf'))

### Now to convert into database elements (tags)